In [240]:
# import glob
import sys
import numpy as np
import pandas as pd
import os
import time
import pickle
import collections
import dgl
from dgl.data.utils import save_graphs,load_graphs
import torch
from datetime import date,timedelta
print(os.getcwd())

import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


RAWDATA = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Afghanistan.csv'
RAWDATA = '~/data/ACLED/country-jul23/2015-01-01-2021-07-23-Yemen.csv'
# RAWDATA = '~/data/ACLED/country-jul23/2017-01-01-2021-07-23-Syria.csv'
WINDOW = 10
HORIZON=1
PREDWINDOW=3

DATASET = RAWDATA.split('-')[7][:-4]
print(DATASET,'dataset_name')

filename = RAWDATA.split('/')[-1]
print('path',RAWDATA,'filename',filename)
start_year = int(filename.split('-')[0])
start_month = int(filename.split('-')[1])
start_day = int(filename.split('-')[2])
end_year = int(filename.split('-')[3])
end_month = int(filename.split('-')[4])
end_day = int(filename.split('-')[5])


path = '../data/{}/'.format(DATASET)
os.makedirs(path, exist_ok=True)
print('path',path)


df = pd.read_csv(RAWDATA,sep=';')
df = df.drop_duplicates(subset=['data_id'], keep='first')
df['event_date'] = pd.to_datetime(df['event_date'])

df.head()
df.loc[df['event_date']=='2021-01-01']

/data/sdeng/workspace/event-ctft/presrc
Yemen dataset_name
path ~/data/ACLED/country-jul23/2015-01-01-2021-07-23-Yemen.csv filename 2015-01-01-2021-07-23-Yemen.csv
path ../data/Yemen/


[nltk_data] Downloading package stopwords to /home/sdeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sdeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/sdeng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/sdeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
4936,7449867,887,YEM61157,61157,2021-01-01,2021,1,Battles,Armed clash,Al Lahaq Tribal Militia (Yemen),...,Ahwar,13.5202,46.7137,1,Aden al Ghad,Subnational,"On 1 January 2021, clashes broke out between m...",1,1610388148,YEM
4937,7450126,887,YEM61158,61158,2021-01-01,2021,2,Battles,Armed clash,Military Forces of Yemen (2016-) Supreme Polit...,...,Al Bayda,13.9852,45.5727,3,Twitter,New media,"Around 1 January 2021 (as reported), AQAP snip...",0,1610388151,YEM
4938,7449872,887,YEM61168,61168,2021-01-01,2021,1,Battles,Government regains territory,Military Forces of Yemen (2012-),...,Al Salib,14.9963,45.1912,2,Aden al Ghad,National,"On 1 January 2021, pro-Hadi military forces ca...",10,1610388148,YEM
4939,7450158,887,YEM61171,61171,2021-01-01,2021,1,Battles,Armed clash,Al Qahih Tribal Militia (Yemen),...,Ataq,14.5377,46.8319,1,Twitter,New media,"On 1 January 2021, members of the Al Qahih tri...",0,1610388151,YEM
4940,7450160,887,YEM61169,61169,2021-01-01,2021,1,Explosions/Remote violence,Remote explosive/landmine/IED,Unidentified Armed Group (Yemen),...,Al Arsh,14.5957,45.1836,1,Twitter,New media,"On 1 January 2021, an explosive device set by ...",10,1610388151,YEM
4941,7449705,887,YEM61161,61161,2021-01-01,2021,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Yemen (2016-) Supreme Polit...,...,Manzar,14.7295,42.9644,1,Al Amaliqah; Yemen Data Project,Local partner-National,"On 1 January 2021, Houthi forces shelled resid...",0,1610388146,YEM
4942,7449706,887,YEM61162,61162,2021-01-01,2021,1,Explosions/Remote violence,Air/drone strike,Operation Restoring Hope,...,As Salif,15.3076,42.6711,2,Al Masirah; Yemen Data Project,Local partner-National,"On 1 January 2021, the Saudi-led coalition con...",0,1610388146,YEM
4943,7449707,887,YEM61163,61163,2021-01-01,2021,1,Battles,Armed clash,Military Forces of Yemen (2016-) Supreme Polit...,...,At Tuhayta,14.1836,43.2264,2,Yemen Data Project; Al Amaliqah,Local partner-National,"On 1 January 2021, Houthi forces clashed with ...",5,1610388146,YEM
4944,7449708,887,YEM61164,61164,2021-01-01,2021,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Yemen (2016-) Supreme Polit...,...,Bayt Maghari,13.9363,43.4555,2,Al Amaliqah; Yemen Data Project,Local partner-National,"On 1 January 2021, Houthi forces shelled resid...",0,1610388146,YEM
4945,7449709,887,YEM61165,61165,2021-01-01,2021,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Yemen (2016-) Supreme Polit...,...,Hays,13.9317,43.4831,2,Al Amaliqah; Yemen Data Project,Local partner-National,"On 1 January 2021, Houthi forces shelled resid...",0,1610388146,YEM


In [241]:
DELTA = 1
# date and get protest count
start = date(start_year,start_month,start_day)
end = date(end_year,end_month,end_day)
delta = timedelta(days=DELTA)
date_table = {}

i = 0
while start <= end:
    start_date_str = start.strftime("%Y-%m-%d")
    date_table[start_date_str] = i
    i += 1
    start += delta
print(len(date_table))

2396


In [242]:
subevents_df = pd.read_csv(path + "subevent2id.txt",names=['id','name'],sep='\t')
subevents = subevents_df['name'].unique()
subevents.sort()
# subevents

In [243]:
subevent_count_dict = {}
for v in subevents:
    subevent_count_dict[v] = np.array([0 for i in range(len(date_table))])
start = date(start_year,start_month,start_day)
end = date(end_year,end_month,end_day)
print('start',start,'end',end)
dayi = 0
while start <= end:
    start_date_str = start.strftime("%Y-%m-%d")
    df_day = df.loc[df['event_date'] == start_date_str]
#     print(start_date_str,'start_date_str',len(df_day))
    df_count = df_day['sub_event_type'].value_counts().rename_axis('unique_values').reset_index(name='counts')
#     print(df_count)
    for i,row in df_count.iterrows():
        subevent_count_dict[row['unique_values']][dayi] = row['counts']
    dayi += 1
    start += delta

Protests_count = subevent_count_dict['Protest with intervention'] + subevent_count_dict['Peaceful protest'] + subevent_count_dict['Excessive force against protesters']

print(Protests_count.shape,'Protests_count')
# print(subevent_count_dict,'subevent_count_dict')
# exit()


start 2015-01-01 end 2021-07-23
(2396,) Protests_count


In [244]:
# subevent_count_dict 
Protests_count.mean()

0.9787145242070117

In [245]:
# build sequence data
subevent_count_seq = []
for k in subevents:
    v = subevent_count_dict[k].tolist()
#     print(v)
    subevent_count_seq.append(v)
 
    
subevent_count_seq = np.array(subevent_count_seq)
subevent_count_seq = np.swapaxes(subevent_count_seq,0,1)
# print(subevent_count_seq.shape,'subevent_count_seq')

# subevent_count_seq[-1]


In [254]:
len(date_ids),len(date_table)
date_table.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [256]:
# get label and Y, and corresponding time
date_ids = list(date_table.values())
date_name = list(date_table.keys())
date_table_rev = dict(zip(date_ids,date_name))
data_time = []
data_Y = []
data_treat = []
data_X = []
data_text = []
for i in range(WINDOW,len(date_ids),HORIZON+PREDWINDOW-1): # no overlap of pre_window
    if i+WINDOW >=len(date_ids) or i+WINDOW+PREDWINDOW-1 >= len(date_ids):
        break
    # treat
    last = subevent_count_seq[i-WINDOW:i]
#     print(i-WINDOW,i,'---',i,i+WINDOW,'   yyy',i+WINDOW,i+WINDOW+PREDWINDOW-1)
    curr = subevent_count_seq[i:i+WINDOW]
    data_X.append(curr)
    treat = curr.sum(0) - last.sum(0)
    data_treat.append(list(np.where(treat>0,1,0)))
    # label
    # print(i+WINDOW,i+WINDOW+PREDWINDOW-1)
    protest = Protests_count[i+WINDOW:i+WINDOW+PREDWINDOW].sum()
    data_Y.append(1 if protest > 0 else 0)
    # time
#     print(i+WINDOW)
    data_time.append(date_ids[i+WINDOW])
    # text
    date_list = [date_table_rev[j] for j in range(i,i+WINDOW)]
    df_window = df.loc[df['event_date'].isin(date_list)]['notes']
    data_text.append(' '.join(df_window.values))

    

# to build counter factual data
data_X = np.stack(data_X)

20
23
26
29
32
35
38
41
44
47
50
53
56
59
62
65
68
71
74
77
80
83
86
89
92
95
98
101
104
107
110
113
116
119
122
125
128
131
134
137
140
143
146
149
152
155
158
161
164
167
170
173
176
179
182
185
188
191
194
197
200
203
206
209
212
215
218
221
224
227
230
233
236
239
242
245
248
251
254
257
260
263
266
269
272
275
278
281
284
287
290
293
296
299
302
305
308
311
314
317
320
323
326
329
332
335
338
341
344
347
350
353
356
359
362
365
368
371
374
377
380
383
386
389
392
395
398
401
404
407
410
413
416
419
422
425
428
431
434
437
440
443
446
449
452
455
458
461
464
467
470
473
476
479
482
485
488
491
494
497
500
503
506
509
512
515
518
521
524
527
530
533
536
539
542
545
548
551
554
557
560
563
566
569
572
575
578
581
584
587
590
593
596
599
602
605
608
611
614
617
620
623
626
629
632
635
638
641
644
647
650
653
656
659
662
665
668
671
674
677
680
683
686
689
692
695
698
701
704
707
710
713
716
719
722
725
728
731
734
737
740
743
746
749
752
755
758
761
764
767
770
773
776
779
782
785
788

In [228]:
print('data_time',len(data_time),'data_Y',len(data_Y),data_X.shape, len(data_text))

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def get_lem_token_list(sentences):
    sentences = re.sub(r'\[[^)]*\]', '',sentences)
    sentences = re.sub(r"[-()*$\"#%/@;:'<>{}`+=~|]", '', sentences)
    sentences = sentences.strip().lower()
    out_tokens = []
    # print(sentences)
    tokens = nltk.word_tokenize(sentences)
    tagged = nltk.pos_tag(tokens)
    # print('tagged:',tagged)
    wordnet_lemmatizer = WordNetLemmatizer()
    for (word,tag) in tagged:
        wntag = get_wordnet_pos(tag)
        if wntag is None:
            w = wordnet_lemmatizer.lemmatize(word)
        else:
            w = wordnet_lemmatizer.lemmatize(word, pos=wntag)
        if w not in [',','.','?','!'] and w not in stopwords.words('english') and w.isalpha() : # did notremove adv and other words
            out_tokens.append(w)
    return out_tokens
def get_sep_token_list(sentences):
    sentences = re.sub(r'\[[^)]*\]', '',sentences)
    sentences = re.sub(r"[-()*$\"#%/@;:'<>{}`+=~|]", '', sentences)
    sentences = sentences.strip().lower()
    # print(sentences)
    tokens = nltk.word_tokenize(sentences)
    tagged = nltk.pos_tag(tokens)
    # print('tagged:',tagged)
    wordnet_lemmatizer = WordNetLemmatizer()
    out_tokens = []
    sents,sent = [], []
    for (word,tag) in tagged:
        wntag = get_wordnet_pos(tag)
        if wntag is None:
            w = wordnet_lemmatizer.lemmatize(word)
        else:
            w = wordnet_lemmatizer.lemmatize(word, pos=wntag)
        if w not in [',','.','?','!','']:# and w not in stopwords.words('english'):
            sent.append(w)
        elif len(sent) > 0:
            sents.append(sent)
            sent = []
    return sents
data_text[0]

data_time 52 data_Y 52 (52, 10, 24) 52


"DEFUSAL: 20 Jan 2017: Afghan forces discover and defuse 3 roadside bombs in Lashkargah city, Helmand province. 20 Jan 2017: at least 21 militants (unidentified) were killed by Afghan securtiy forces in clashes in Sangin district and Lashkargah city, Helmand province. 3 Pakistanis reported killed but no location provided. 21 fatalities split between 2 locations. 20 Jan 2017: Taliban accuse government forces of an attack which killed 6 civilians in Kapisa province - no corroboration. 20 Jan 2017: Police report attack on a checkpoint in Kapisa province which killed 3 policeman. Taliban blamed but had not claimed responsibility. Reported 20 Jan 2017: 27 Taliban killed, 17 wounded, and 16 police killed or wounded in a series of clashes in Maywand district, Kandahar province. Recorded 27 Taliban fatalities and 8 police deaths. 20 Jan 2017: 2 killed - one police and one suspected Taliban militant - in a clash in Jardoq area, Qush Tepa district, Jowazjan province. Taliban had not yet commente

In [168]:
# with open('/home/sdeng/data/stopwords-en-basic.txt','r') as f:
#     stop_words = f.read().splitlines()
# stop_words

get_sep_token_list(data_text[0])[-22]

['on', '11', 'january', '2017', 'late']

In [227]:
def movingaverage(a, n=3) :
    padding = []
    for i in range(n-1):
        padding.append(a[:i+1].mean())
    padding = np.array(padding)
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return np.concatenate((padding, ret[n - 1:] / n),0)
data_X_smooth = [] # 549,10,24
d1,d2,d3 = data_X.shape
for i in range(d1):
    tmp = []
    for j in range(d2):
        tmp.append(list(movingaverage(data_X[i,j,:],WINDOW//2)))
    data_X_smooth.append(tmp)
data_X_smooth = np.array(data_X_smooth)
print('data_X_smooth',data_X_smooth.shape)

data_X_smooth (52, 10, 24)


In [216]:
# from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS, original_scorer

# all_sent_list = []
# for v in data_text:
#     sent_list = get_sep_token_list(v)
#     all_sent_list += sent_list



In [217]:
# print(len(all_sent_list))
# all_sent_list[0]

In [218]:
# ignore_set = ENGLISH_CONNECTOR_WORDS
# phrases = Phrases(all_sent_list, min_count=5, threshold=0.2, scoring="npmi", connector_words=ignore_set)



In [229]:
from sklearn.feature_extraction.text import TfidfVectorizer

with open('/home/sdeng/data/stopwords-en-basic.txt','r') as f:
    stop_words = f.read().splitlines()
stop_words += ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 'placeholder', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn']
# vectorizer = TfidfVectorizer(stop_words=stop_words,min_df=0.05,max_df=0.95)
# 
stop_words += ['20','3','21','2','2017','6','7','8','9','10','11','12','13','14','15','16','17','18','19','22','23','24','25','26','27','28','29','30','31','2015','2016','2018','2019','2020','2021']

all_tokens = []
for sentences in data_text:
    tokens = get_lem_token_list(sentences)
    all_tokens.append(tokens)

In [230]:

vectorizer = TfidfVectorizer(tokenizer=(lambda x:x), lowercase=False, stop_words=stop_words,min_df=5,max_df=0.99)#token_pattern=r'(?u)\b\w*[a-zA-Z]\w*\b') # stopwords='english' u'(?u)\b\w*[a-zA-Z]\w*\b

tfidf = vectorizer.fit_transform(all_tokens)
feature_names = vectorizer.get_feature_names()

/home/sdeng/anaconda2/envs/jupyter/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", '/', '0', '1', '4', '5', '<', '>', 'A', 'C', 'D', 'E', 'H', 'L', 'O', 'P', 'R', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [231]:
len(feature_names)
tfidf_vocab = path + 'tfidf_vocab.txt'
tfidf_vocab_f = open(tfidf_vocab, 'w')
for i in range(len(feature_names)):
    tfidf_vocab_f.write("{}\n".format(feature_names[i]))
tfidf_vocab_f.close()

In [234]:
# '9am'.isalpha() 
tfidf.shape
with open(path+'tmp_label.pkl','wb') as f:
    pickle.dump([data_time,data_Y,data_X_smooth,tfidf],f)

In [235]:
with open(path+'tmp_label.pkl','rb') as f:
    [data_time,data_Y,data_X_smooth,tfidf] = pickle.load(f)

In [260]:
data_treat = np.array(data_treat)
data_treat.shape
len(data_Y)

792